<a href="https://colab.research.google.com/github/shibin-CR7/FOXIOM/blob/main/resume_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pgvector --quiet
%pip install langchain langchain-community --quiet
%pip install chromadb --quiet
%pip install sentence-transformers --quiet
%pip install langchain-fireworks --quiet
%pip install pypdf --quiet
%pip install sentence_transformers--quiet
%pip install chromadb --quiet
%pip install pypdf2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.8 MB/s et

In [ ]:
!unzip /content/HR_executive.zip -d /content

Archive:  /content/HR_executive.zip
  inflating: /content/HR executive/_SRILAKSHMIRATNAKARNCV.pdf  
  inflating: /content/HR executive/AISHWARYA V.pdf  
  inflating: /content/HR executive/AJUN SAJI 123.pdf  
  inflating: /content/HR executive/Akshay Divakar CV-1.pdf  
  inflating: /content/HR executive/Amrutha_R_HR_Resume.pdf  
  inflating: /content/HR executive/Anjaly Resume..pdf  
  inflating: /content/HR executive/anjitha vp CV.pdf  
  inflating: /content/HR executive/Anooj_CV-2.pdf  
  inflating: /content/HR executive/CV NEW (1).pdf  
  inflating: /content/HR executive/DOC-20240422-WA0114_.pdf  
  inflating: /content/HR executive/DOC-20240515-WA0009._20240521_081510_0000.pdf  
  inflating: /content/HR executive/gayathri cv resume.pdf  
  inflating: /content/HR executive/Gopika Updated Resumee .pdf  
  inflating: /content/HR executive/HR CV.pdf  
  inflating: /content/HR executive/IMG_20240107_233244-compressed.pdf  
  inflating: /content/HR executive/Liyan Varghese CV.pdf  
  infla

In [ ]:
#!export OPENAPI_API_KEY=WAtTqN0IdVMKsd8C5Zs9ixwHApIpTvJyzmiGujazA9ya1Sxf
!export fireworks_api_key="WAtTqN0IdVMKsd8C5Zs9ixwHApIpTvJyzmiGujazA9ya1Sxf"


In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel

In [ ]:
import os
os.environ['FIREWORKS_API_KEY'] = '6byKKVHFMZdwFFLcJqcYAkEGTVkKJ8HfqTd2SP0Z5XYZaDdS'


In [ ]:
import os
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader

# Define a function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Directory containing PDFs
pdf_directory = "./HR executive"

# Load documents from the directory
loader = PyPDFDirectoryLoader(pdf_directory)
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

# Extract text and store with filenames
all_texts = []
for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):
       pdf_path = os.path.join(pdf_directory, filename)
       text = extract_text_from_pdf(pdf_path)
       all_texts.append({"text": text, "file_name": filename})

#Example: Display the first split chunk and the first extracted text with filename
print(splits[0])
#print(all_texts[0])


page_content='Contact Details  
 
Email : sreelakshmimidhila@gmail.com  
Phone No: 9747321465  
Location:  Kochi  
Linkedin:  
https://www.linkedin.com/in/sreelaks
hmi-s-9179aa195  
 
SKILLS  
• Talent Acquisition  
• Recruitment  
• Candidate Screening and sourcing 
through Boolean search and X-ray 
search  
• Marketing Research using SPSS 
Software.  
• Onboarding Process  
• Public Relations  
• Skilled in designing vis ually 
appealing  & interactive dashboards  
to communicate  insights and trends 
effectively  
 
PROJECTS  
• Is OTT a disruption for Movie 
theatre Industry  
• Power BI - Supply chain 
Management  
• Crop Protection and Irrigation 
System  
• IoT based Home automation System  
 
ACCOMPLISHMENTS  
• NSS Volunteer  
• Professional Student Summit  
• College Union Member  
 
   SR E E L A K S H M I  S  
   H R  A S S O C I A T E' metadata={'source': 'HR executive/Sreelakshmi S. Resume.pdf', 'page': 0}


In [ ]:
import chromadb
from chromadb.config import Settings
client = chromadb.Client(Settings())
collection = client.create_collection(name="HR_docs")


In [ ]:
for idx, split in enumerate(splits):
    collection.add(
        documents=[split.page_content],
        metadatas=[{"filename": split.metadata["source"]}],
        ids=[f"split_{idx}"]
    )

result = collection.get(["split_0"])


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 75.3MiB/s]


In [ ]:
from langchain.vectorstores import Chroma
# Initialize Chroma retriever
retriever = Chroma(client=client, collection_name="HR_docs")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
from langchain_fireworks import Fireworks
llm = Fireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    temperature=0.0,
    max_tokens=2000,
    top_k=1,
)

In [ ]:
job_description = """
We are seeking a motivated HR Executive with  1-2 years of experience, preferably from an IT background.
"""
question = "Please extract and format the candidate details as specified."

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Updated template to include the text chunk
template = """
You are an AI language model assistant. Your task is to evaluate a pool of resumes against the following job description (JD) and collect specific details about each candidate to determine their suitability.

Job Description (JD):
{job_description}

Resume:
{text_chunk}

For each candidate, please extract the following information in the specified format:

name: <candidate name>
contact:
    email: <email>
    phone: <phone number>2
experience:
    1. <role_name1> - <start_date1> to <end_date1>
    2. <role_name2> - <start_date2> to <end_date2>
    3. <role_name3> - <start_date3> to <end_date3>
skills: <skill1>, <skill2>, <skill3>, ...

question: {question}
"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

# Function to format input using the prompt template
def format_input(job_description, question, text_chunk):
    return prompt_perspectives.format(job_description=job_description, question=question, text_chunk=text_chunk)

# Initialize an empty list to store candidate details
candidate_details = []

# Process each text chunk with the LLM
for split in splits:
    input_text = format_input(job_description, question, split.page_content)
    response = llm(input_text)
    candidate_details.append(response)

# Display the first set of candidate details
print(candidate_details[0])


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



answer:

name: SR E E L A K S H M I  S
contact:
    email: sreelakshmimidhila@gmail.com
    phone: 9747321465
experience:
   1. Not specified in the resume
skills: 
   1. Talent Acquisition
   2. Recruitment
   3. Candidate Screening and sourcing through Boolean search and X-ray search
   4. Marketing Research using SPSS Software
   5. Onboarding Process
   6. Public Relations
   7. Skilled in designing visually appealing & interactive dashboards to communicate insights and trends effectively
